# Proyecto 2
### Data science
- Abner Gardia 21285
- Esteban Donis 21610
- Daniel Gomez 21429

In [82]:
# project 2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from spellchecker import SpellChecker
import re
from sklearn.preprocessing import LabelEncoder
import pycountry
import names
import nltk
from nltk.stem import WordNetLemmatizer

### Inciso 2
Analice el problema planteado y los datos.

In [83]:
train = pd.read_csv('./data/train.csv')

In [84]:
train.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


In [85]:
train.describe()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
count,36765,36765,36765,36765,36765
unique,36765,4191,36691,7,3
top,0013cc385424,91B1F82B2CF1,Summer projects should be student-designed,Evidence,Adequate
freq,1,23,14,12105,20977


In [86]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36765 entries, 0 to 36764
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   discourse_id             36765 non-null  object
 1   essay_id                 36765 non-null  object
 2   discourse_text           36765 non-null  object
 3   discourse_type           36765 non-null  object
 4   discourse_effectiveness  36765 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB


### Inciso 3
Describa las tareas de limpieza y preprocesamiento que llevó a cabo.

### Cambios columna discourse_text

Reemplazar abreviaciones

In [87]:
train['discourse_text'] = train['discourse_text'].str.replace(r"n't", ' not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"'s", ' is', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"its", 'it is', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"'re", ' are', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"doesn", 'does not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"don", 'do not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"didn", 'did not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"won", 'will not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"can't", 'can not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"alot", 'a lot', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"what's", 'what is', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"thats", 'that is', regex=True)

Eliminar cualquier símbolo que no sea una letra o número

In [88]:
train['discourse_text'] = train['discourse_text'].str.replace(r"[^a-z0-9A-Z ]", ' ', regex=True)

Reemplazar palabras como dont, isnt, etc. Que no estén escritas con apostrofes

In [89]:
train['discourse_text'] = train['discourse_text'].str.replace(r"dont", 'do not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"isnt", 'is not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"wont", 'will not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"werent", 'were not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"hasnt", 'has not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"havent", 'have not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"shouldnt", 'should not', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"couldnt", 'could not', regex=True)

Transformar todas las palabras a minúsculas

In [90]:
train['discourse_text'] = train['discourse_text'].str.lower()

Obtención paises para comparar en corrección de errores ortográficos

In [91]:
country_names = [country.name.lower() for country in pycountry.countries]

print(country_names)

['aruba', 'afghanistan', 'angola', 'anguilla', 'åland islands', 'albania', 'andorra', 'united arab emirates', 'argentina', 'armenia', 'american samoa', 'antarctica', 'french southern territories', 'antigua and barbuda', 'australia', 'austria', 'azerbaijan', 'burundi', 'belgium', 'benin', 'bonaire, sint eustatius and saba', 'burkina faso', 'bangladesh', 'bulgaria', 'bahrain', 'bahamas', 'bosnia and herzegovina', 'saint barthélemy', 'belarus', 'belize', 'bermuda', 'bolivia, plurinational state of', 'brazil', 'barbados', 'brunei darussalam', 'bhutan', 'bouvet island', 'botswana', 'central african republic', 'canada', 'cocos (keeling) islands', 'switzerland', 'chile', 'china', "côte d'ivoire", 'cameroon', 'congo, the democratic republic of the', 'congo', 'cook islands', 'colombia', 'comoros', 'cabo verde', 'costa rica', 'cuba', 'curaçao', 'christmas island', 'cayman islands', 'cyprus', 'czechia', 'germany', 'djibouti', 'dominica', 'denmark', 'dominican republic', 'algeria', 'ecuador', 'egy

Obtención nombres para comparar en corrección de errores ortográficos

In [92]:
namesToCompare = {names.get_first_name().lower() for _ in range(1000)}

Palabras aleatorias que debe saltarse

In [93]:
wordsToAvoid = ['bogota', 'americans', 'american', 'leonardo', 'vinci', 'paris', 'europe', 'facs', 'huag', 'beckman', 'venus', 'bugatti', 'nasa', 'beijing', 'colombian', 'colombians', 'mr', 'mrs', 'amd', 'columbian', 'arturo', 'obama', 'venice', 'mona', 'pc', 'andrew', 'april', 'youtube', 'twitter', 'eckman', 'ufo', 'apps']

Corrector de errores de escritura

In [94]:
spell = SpellChecker()

def wordCorrection(text):
    words = text.split()
    misspelled = spell.unknown(words)

    for word in misspelled:
        if spell.correction(word) is None:
            continue
        if word in wordsToAvoid:
            continue
        print(word, spell.correction(word))
        text = text.replace(word, spell.correction(word))

    return text

In [95]:
# train['discourse_text'] = train['discourse_text'].apply(wordCorrection)

Eliminar stop words del texto

In [96]:
stopWordsEnglish = set(stopwords.words('english'))

def deleteStopWords(text):
    words = text.split()

    processed_words = [word for word in words if word not in stopWordsEnglish]

    return ' '.join(processed_words)

In [97]:
train['discourse_text'] = train['discourse_text'].apply(deleteStopWords)

### Cambios columna discourse_type

In [98]:
train['discourse_type'] = train['discourse_type'].str.lower()
train['discourse_type'] = train['discourse_type'].str.replace(r"[^a-z0-9 ]", ' ', regex=True)

### Cambios columna discourse_effectiveness

In [99]:
train['discourse_effectiveness'] = train['discourse_effectiveness'].str.lower()
train['discourse_effectiveness'] = train['discourse_effectiveness'].str.replace(r"[^a-z0-9 ]", ' ', regex=True)

#### Delete duplicated rows

In [100]:
train = train.drop_duplicates()

#### Normalize text data

In [101]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    return text.strip()

train['discourse_text'] = train['discourse_text'].apply(normalize_text)

In [ ]:
train.to_csv('result/data_cleaned.csv', index=False)